<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/DCGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install tensorflow pillow numpy matplotlib scikit-image opencv-python

In [10]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload your image

Saving Mona_Lisa.jpg to Mona_Lisa.jpg


In [7]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import cv2

# Step 1: Load and prepare data
def load_image(image_path, size=(256, 256)):
    """Load and resize image"""
    img = Image.open(image_path)
    img = img.resize(size)
    img = np.array(img) / 255.0  # Normalize to [0,1]
    return img

def load_dataset(directory, size=(256, 256)):
    """Load all images from directory"""
    images = []
    for filename in os.listdir(directory):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(directory, filename)
            img = load_image(img_path, size)
            images.append(img)
    return np.array(images)

# Step 2: Preprocess images
def preprocess_images(images):
    """Preprocess images for the model"""
    # Convert to float32
    images = images.astype('float32')
    # Ensure values are in [-1, 1]
    images = (images * 2) - 1
    return images

# Step 3: Create mask
def create_random_mask(shape, max_boxes=5):
    """Create random rectangular masks"""
    mask = np.ones(shape)
    for _ in range(np.random.randint(1, max_boxes)):
        # Random box coordinates
        x1, x2 = sorted(np.random.randint(0, shape[1], 2))
        y1, y2 = sorted(np.random.randint(0, shape[0], 2))
        mask[y1:y2, x1:x2] = 0
    return mask

# Step 4: Define Generator and Discriminator
def build_generator():
    """Create the generator model"""
    model = tf.keras.Sequential([
        # Encoder
        layers.Input(shape=(256, 256, 3)),
        layers.Conv2D(64, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(256, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),

        # Decoder
        layers.Conv2DTranspose(128, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(64, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')
    ])
    return model

def build_discriminator():
    """Create the discriminator model"""
    model = tf.keras.Sequential([
        layers.Input(shape=(256, 256, 3)),
        layers.Conv2D(64, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(128, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Conv2D(256, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Step 5: Combine into DCGAN
class DCGAN(tf.keras.Model):
    def __init__(self):
        super(DCGAN, self).__init__()
        self.generator = build_generator()
        self.discriminator = build_discriminator()

        # Optimizers
        self.gen_optimizer = tf.keras.optimizers.Adam(1e-4)
        self.disc_optimizer = tf.keras.optimizers.Adam(1e-4)

        # Loss
        self.cross_entropy = tf.keras.losses.BinaryCrossentropy()
        self.mse = tf.keras.losses.MeanSquaredError()

    @tf.function
    def train_step(self, images, masks):
        """Single training step"""
        batch_size = tf.shape(images)[0]

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            # Generate images
            masked_images = images * masks
            generated_images = self.generator(masked_images, training=True)

            # Combine real and generated images
            filled_images = masked_images + (1 - masks) * generated_images

            # Train discriminator
            real_output = self.discriminator(images, training=True)
            fake_output = self.discriminator(filled_images, training=True)

            # Calculate losses
            gen_loss = self.mse(images, filled_images)
            disc_loss = self.cross_entropy(tf.ones_like(real_output), real_output) + \
                       self.cross_entropy(tf.zeros_like(fake_output), fake_output)

        # Apply gradients
        gen_gradients = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        disc_gradients = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.gen_optimizer.apply_gradients(zip(gen_gradients, self.generator.trainable_variables))
        self.disc_optimizer.apply_gradients(zip(disc_gradients, self.discriminator.trainable_variables))

        return gen_loss, disc_loss

# Step 6: Training function
def train_model(model, dataset, epochs=100, batch_size=32):
    """Train the model"""
    for epoch in range(epochs):
        for batch in range(0, len(dataset), batch_size):
            # Get batch of images
            batch_images = dataset[batch:batch + batch_size]

            # Create masks for this batch
            masks = np.stack([create_random_mask((256, 256)) for _ in range(len(batch_images))])
            masks = np.expand_dims(masks, axis=-1)
            masks = np.repeat(masks, 3, axis=-1)

            # Train on batch
            gen_loss, disc_loss = model.train_step(batch_images, masks)

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: Generator Loss = {gen_loss:.4f}, Discriminator Loss = {disc_loss:.4f}")

# Step 7: Evaluation functions
def evaluate_model(model, test_image, test_mask):
    """Evaluate model on a single test image"""
    # Generate inpainted image
    masked_image = test_image * test_mask
    generated = model.generator(np.expand_dims(masked_image, 0), training=False)
    generated = generated[0].numpy()

    # Combine masked and generated portions
    inpainted = masked_image + (1 - test_mask) * generated

    # Calculate metrics
    psnr_value = psnr(test_image, inpainted)
    ssim_value = ssim(test_image, inpainted, multichannel=True)

    return inpainted, psnr_value, ssim_value

# Step 8: Visualization function
def visualize_results(original, masked, inpainted):
    """Visualize original, masked, and inpainted images"""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(original)
    axes[0].set_title('Original')
    axes[0].axis('off')

    axes[1].imshow(masked)
    axes[1].set_title('Masked')
    axes[1].axis('off')

    axes[2].imshow(inpainted)
    axes[2].set_title('Inpainted')
    axes[2].axis('off')

    plt.show()

def main():
    # Load dataset
    # Load the single image
    image_path = "Mona_Lisa.jpg"  # Updated path

    # Ensure the file exists
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Could not find image at {image_path}")

    # Load and prepare single image
    image = load_image(image_path)
    images = np.expand_dims(image, axis=0)  # Add batch dimension
    images = preprocess_images(images)

    # Create and train model
    model = DCGAN()

    # Print model summary to verify architecture
    print("Generator Summary:")
    model.generator.summary()
    print("\nDiscriminator Summary:")
    model.discriminator.summary()

    # Train with error handling
    try:
        train_model(model, images, epochs=100, batch_size=1)  # Use batch_size=1 since we have one image
    except Exception as e:
        print(f"Training error: {str(e)}")
        return

    # Evaluate on test image
    try:
        test_image = load_image(image_path)
        test_mask = create_random_mask((256, 256))
        test_mask = np.expand_dims(test_mask, axis=-1)
        test_mask = np.repeat(test_mask, 3, axis=-1)

        # Get results
        inpainted, psnr_value, ssim_value = evaluate_model(model, test_image, test_mask)

        # Print metrics
        print(f"PSNR: {psnr_value:.2f}")
        print(f"SSIM: {ssim_value:.2f}")

        # Visualize results
        masked_image = test_image * test_mask
        visualize_results(test_image, masked_image, inpainted)

    except Exception as e:
        print(f"Evaluation error: {str(e)}")
if __name__ == "__main__":
    main()

TypeError: in user code:

    File "<ipython-input-7-5c8a6eff74f2>", line 108, in train_step  *
        masked_images = images * masks

    TypeError: Input 'y' of 'Mul' Op has type float64 that does not match type float32 of argument 'x'.
